# Imports 

In [1]:
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [2]:
import os 
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
from time import strftime

In [4]:
tf.compat.v1.disable_eager_execution()

In [5]:
from PIL import Image

# Constants

In [6]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
COLOR_CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT*IMAGE_WIDTH*COLOR_CHANNELS

# Get the data

In [7]:
%%time 
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',',dtype=int)

Wall time: 311 ms


In [8]:
y_train_all.shape

(60000,)

In [9]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',',dtype=int)

In [10]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int) 

Wall time: 41.9 s


In [11]:
%%time 
x_test = np.loadtxt(X_TEST_PATH, delimiter=',',dtype='int')

Wall time: 4.67 s


# Explore

In [12]:
x_train_all.shape

(60000, 784)

In [13]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [14]:
y_train_all.shape

(60000,)

In [15]:
x_test.shape

(10000, 784)

In [16]:
y_train_all[0]

5

In [17]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [18]:
# Re-scale 
x_train_all, x_test = x_train_all/255.0, x_test/255.0 #dividing by 255.0 to scale the outputs and to convert the data into a float
#converting the data to a float helps us to ease the calculation 

# Convert target values to One Hot Encoding

In [19]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [20]:
y_train_all.shape

(60000, 10)

In [21]:
y_test = np.eye(NR_CLASSES)[y_test]

In [22]:
y_test.shape

(10000, 10)

# Create validation dataset from training data

In [23]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [24]:
x_train.shape

(50000, 784)

In [25]:
x_val.shape

(10000, 784)

Tensors are a generalization of arrays to a higher dimensions <br>
Tensors follow rules of linear algebra

# Setup Tensorflow Graph

In [26]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS],name='X')
# we put none in the shape because the first dimension will hold the number of samples that we pass and we can change the batch size later on while training and we put 784 in the shape because we have 784 features in an image 
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES],name='Labels')

# Neural Network Architecture

Input for the network = Each row of the X_train matrix <br>
Shape of weights of layer 1 = (shape of input of network, n_layer_1)<br>
Input for layer 1 = (Input for the network $\times$ weights_layer_1) + bias_layer_1<br>
Output for layer 1 = relu(input for layer 1)<br>
<br>
Input for layer 2 = output of layer 1<br> 
Shape of weights of layer 2 = (shape of output of layer 1, n_layer_2)<br>
Input of layer 2 = (Output of layer 1 $\times$ weights_layer_2) + bias_layer_2<br>
Output of layer 2 = relu(input for layer 2)<br>
<br>
Input for layer 3 = output of layer 2 <br>
Shape of weights of layer 3 = (shape of output of layer 2, n_layer_3)<br>
Input of layer 3 = (output of layer 2 $\times$ weights_layer_3) + bias_layer_3<br>
Output of layer 3 = softmax(input of layer 3)

Input for any layer = (output for previous layer * weights for current layer) + bias for current layer<br>
weights for any layer = trucnated normal distribution of shape (n_neurons in previous layer, n_neurons in current layer)<br>
bias for any layer = an array of 0 with shape n_neurons in current layer<br>
<br>
If the layer is first layer, previous layer = input to the network 

### Hyperparamters

In [27]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [28]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)
        return layer_out

In [29]:
#model without dropout
# layer_1 = setup_layer(input=X, weight_dim=[TOTAL_INPUTS,n_hidden1],bias_dim=[n_hidden1],name='layer_1')
# layer_2 = setup_layer(input=layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2],name='layer_2')
# layer_3 = setup_layer(input=layer_2, weight_dim=[n_hidden2, NR_CLASSES],bias_dim=[NR_CLASSES],name='out')
# model_name = f'{n_hidden1}-{n_hidden2} LR {learning_rate} E{nr_epochs}'

In [30]:
layer_1 = setup_layer(input=X, weight_dim=[TOTAL_INPUTS,n_hidden1],bias_dim=[n_hidden1],name='layer_1')
layer_drop = tf.nn.dropout(layer_1,keep_prob=0.8,name='dropout_layer')
layer_2 = setup_layer(input=layer_drop, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2],name='layer_2')
layer_3 = setup_layer(input=layer_2, weight_dim=[n_hidden2, NR_CLASSES],bias_dim=[NR_CLASSES],name='out')
model_name = f'{n_hidden1}-{n_hidden2}-Dropout- LR {learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Loss, Optimization and metrics

### Defining loss function 

### Tensorboard Setup

In [31]:
#folder for tensorboard
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH,folder_name)
try:
    os.makedirs(directory)
except OSError as error:
    print(error.strerror)
else:
    print('Successfully created directory')

Successfully created directory


In [32]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=layer_3)) 
    #the actual value for comparison is Y and the predicted value is output of the network for calculatig losses 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Defining Optimizer

In [33]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) #initialize the adam optimizer
    train_step = optimizer.minimize(loss=loss) #use the optimizer to minimize loss

### Accuracy metric

In [34]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(layer_3,axis=1), tf.argmax(Y,axis=1)) #get the number of correct predictions by comparing y to yhat
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) #calculate the accuracy by taking mean of all epochs 

In [35]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy',accuracy) #add a scalar to the summary by the name of accuracy which stores accuracy in it
    tf.summary.scalar('Cost',loss) #add a scalar to the summary by the name of cost which stores the loss in it 
    # We add these scalars so that we can see them in tensorboard 

# Check input images in tensorboard

In [36]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X,[-1,IMAGE_HEIGHT,IMAGE_WIDTH,COLOR_CHANNELS])
    tf.summary.image('Image_input',x_image,max_outputs=4)

# Run session

In [37]:
sess = tf.Session() #create a tensorflow session 

### Setup File Writer and merge summaries

In [38]:
merged_summary = tf.summary.merge_all() #create a merged summary for visualization 
train_writer = tf.summary.FileWriter(directory+'/train') #initialize a filewriter which dumps the data from the summary to the train folder 
train_writer.add_graph(sess.graph) #add the outputs from train writer to the tensorboard graph 

validation_writer = tf.summary.FileWriter(directory+'/validation') #create another file writer for validation dataset 


# Initialize variables

In [39]:
# Initializing all the variables 
init = tf.global_variables_initializer()
sess.run(init)

In [40]:
size_of_batch = 1000
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [41]:
def next_batch(batch_size, data, labels): #create a function which helps us to break the data into batches of batch size 
    
    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch>num_examples:
        start = 0
        index_in_epoch = batch_size

    end = index_in_epoch
    return data[start:end], labels[start:end]

### Training loop

In [42]:
for epoch in range(nr_epochs): #train our network 
    #Training DataSet
    for i in range(nr_iterations):
        batch_x, batch_y = next_batch(batch_size=size_of_batch,data=x_train, labels=y_train) #get the x and y for the batches 
        feed_dictionary = {X:batch_x, Y:batch_y} #create a feed dictionary which contains data for training which has the placeholders holding our data
        sess.run(train_step, feed_dict = feed_dictionary) #run the session by feeding the optimizer (train_step) and the data dictionary
    s,batch_accuracy = sess.run(fetches=[merged_summary,accuracy], feed_dict=feed_dictionary) #obtain the output from the training epoch 
    #which fetches the summary and the accuracy from the epoch after training on the data 
    train_writer.add_summary(s,epoch) #use our training set file writer to add the summary to our disk after each epoch so we can visualize it on tensorboard
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}') #print the accuracy after each epoch 
    #Validation DataSet
    summary = sess.run(fetches=merged_summary,feed_dict={X:x_val, Y:y_val}) #train the validation set in our neural network and fetch the summary from each epoch
    #after training on validation dataset where the X placeholder stores x_val and Y placeholder stores y_val
    validation_writer.add_summary(summary,epoch) #use the validation file writer to write the summary at each epoch to our disk so that we can visualize it on tensorboard
print('Done Training') #print Done Training after the training has finished 

Epoch 0 	| Training Accuracy = 0.8500000238418579
Epoch 1 	| Training Accuracy = 0.8629999756813049
Epoch 2 	| Training Accuracy = 0.8669999837875366
Epoch 3 	| Training Accuracy = 0.8679999709129333
Epoch 4 	| Training Accuracy = 0.8700000047683716
Epoch 5 	| Training Accuracy = 0.8730000257492065
Epoch 6 	| Training Accuracy = 0.9750000238418579
Epoch 7 	| Training Accuracy = 0.9800000190734863
Epoch 8 	| Training Accuracy = 0.9800000190734863
Epoch 9 	| Training Accuracy = 0.984000027179718
Epoch 10 	| Training Accuracy = 0.9829999804496765
Epoch 11 	| Training Accuracy = 0.9869999885559082
Epoch 12 	| Training Accuracy = 0.9879999756813049
Epoch 13 	| Training Accuracy = 0.9890000224113464
Epoch 14 	| Training Accuracy = 0.9900000095367432
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9879999756813049
Epoch 17 	| Training Accuracy = 0.9890000224113464
Epoch 18 	| Training Accuracy = 0.9909999966621399
Epoch 19 	| Training Accuracy = 0.99000000

# Make a prediction

In [43]:
img = Image.open('MNIST/test_img.png')
img

In [44]:
bw = img.convert(mode='L')
img_array = np.invert(bw)

In [45]:
test_img = img_array.ravel()

In [46]:
test_img.shape

(784,)

In [47]:
prediction = sess.run(fetches=tf.argmax(layer_3, axis=1),feed_dict={X:[test_img]})

In [48]:
print(f'Prediction for test image is {prediction[0]}')

Prediction for test image is 2


# Testing and Evaluation

In [49]:
accuracy_test = sess.run(fetches=accuracy,feed_dict={X:x_test, Y:y_test})
print(f'The accuracy on the test set is {accuracy_test:.2%}')

The accuracy on the test set is 97.82%


# Reset for the next run

In [50]:
#close all file writers and sessions and reset the graphs to release all resources
train_writer.close() 
validation_writer.close()
sess.close()
tf.reset_default_graph()